<h1><center>cs1001.py , Tel Aviv University, Spring 2018/19</center></h1>
<img src="http://www.pngall.com/wp-content/uploads/2016/05/Python-Logo-PNG-Image-180x180.png" width=50/>

## Recitation 10

We discussed iterators, generators, and the Karp-Rabin algorithm for string matching 

###### Takeaways:
- A generator function is a function that contains the yield command and returns a genertor object. 


- The Karp-Rabin algorithm is a probabilistic string matching algorithm (find all occurences of a string of length $m$ in a string of length $n$) running in linear time $O(m+n)$ (as opposed to the naive solution which has running time $O(nm)$).
- Make sure you read our KR <a href="http://tau-cs1001-py.wdfiles.com/local--files/recitation-logs-2017b/KR-summary_new.pdf">summary.</a>
- Make sure you understand the way the algorithm works, and in particular the "rolling hash mechanism", that is, how to compute the fingerprint of the next substring in $O(1)$ time, given the fingerprint of the current substring.
- Make sure you understand the "arithmetization" step used by the algorithm.

#### Code for printing several outputs in one cell (not part of the recitation):

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##  Iterators and Generators

### Iterators

In [2]:
l = "123"  #[1,2,3]
li = iter(l)
type(li)
li2 = iter(l)

str_iterator

In [25]:
next(li)

'1'

In [26]:
next(li)

'2'

In [27]:
z = next(li)
print("z is", z)

z is 3


In [28]:
next(li)

StopIteration: 

In [29]:
next(li2)

'1'

In [30]:
for elem in li2:
    print(elem)

2
3


### Generators

In [32]:
def countdown_gen():
    ''' calling it creates an iterator for the values 5,4,3,2,1,'launch' '''
    yield 5
    yield 4
    yield 3
    yield 2
    yield 1
    yield 'launch'

In [33]:
cd = countdown_gen()
cd #generator object (private case of iterator)

<generator object countdown_gen at 0x000002623F0CFE58>

In [34]:
next(cd) 

5

In [35]:
next(cd) 

4

In [36]:
for e in cd:
    print(e)

3
2
1
launch


With generators we can have infinite iterations

In [ ]:
def countdown_infinite():
    i = 0
    while True:
        yield i
        i += 1
        
[i for i in range(10)]

#### Solving this <a href="http://tau-cs1001-py.wdfiles.com/local--files/recitation-logs-2017a/a10_geneators_exam_q.pdf"> exam question about generators</a>

Section (b)

In [37]:
def SomePairs():
    i=1
    while True:
        for j in range(i):
            yield(i,j)
        i=i+1

gen = SomePairs()
[next(gen) for _ in range(10)]

[(1, 0),
 (2, 0),
 (2, 1),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3)]

Section (c)

In [38]:
def RevGen(PairsGen):
    pairs = PairsGen()
    while True:
        pair = next(pairs)
        yield(pair[1],pair[0])

gen = RevGen(SomePairs)
[next(gen) for _ in range(10)]

[(0, 1),
 (0, 2),
 (1, 2),
 (0, 3),
 (1, 3),
 (2, 3),
 (0, 4),
 (1, 4),
 (2, 4),
 (3, 4)]

Section (d1)

In [41]:
def UnionGenerators(gen1, gen2):
    while True:
        yield next(gen1)
        yield next(gen2)
        
ug = UnionGenerators(SomePairs(), RevGen(SomePairs))

for _ in range(10):
    next(ug)

(1, 0)

(0, 1)

(2, 0)

(0, 2)

(2, 1)

(1, 2)

(3, 0)

(0, 3)

(3, 1)

(1, 3)

Section (d2)

In [ ]:
def EqPairs():
    i=0
    while True:
        yield (i,i)
        i=i+1
        
def AllPairs():
    return UnionGenerators(SomePairs(), UnionGenerators(EqPairs(), RevGen(SomePairs)))

gen = AllPairs()
[next(gen) for _ in range(10)]

## The string-matching problem

Given a string $text$ of length $n$, and a short(er) string $pattern$ of length $m$ ($m\leq n$), report all occurrances of $pattern$ in $text$.

Example:

$text = $"abracadabra",  $pattern = $"abr"

The requested output should be $[0,7]$, since $pattern$ appears in $text$ in indices $0,7$.

## Karp-Rabin Algorithm

- The algorithm works as follows:
    - An initial "fingerprint" is computed for the pattern string and the first substring of length $m$ in the text
    - If both fingerprints are equal, we assume a match
    - Then, a "rolling hash" mechanism computes the next fingerprint in $O(1)$ time given the current fingerprint in the text
    - At each stage again a comparison is made and equal fingerprints are considered a match

In [4]:
def fingerprint(text, basis=2**16, r=2**32-3):
    """ used to compute karp-rabin fingerprint of the pattern
        employs Horner method (modulo r) """
    partial_sum = 0
    for ch in text:
        partial_sum =(partial_sum*basis + ord(ch)) % r
    return partial_sum

def text_fingerprint(text, m, basis=2**16, r=2**32-3):
    """ computes karp-rabin fingerprint of the text """
    f=[]
    b_power = pow(basis, m-1, r)
    f.append(fingerprint(text[0:m], basis, r))
    # f[0] equals first text fingerprint 
    for s in range(1, len(text)-m+1):
        new_fingerprint = ( (f[s-1] - ord(text[s-1])*b_power)*basis
                         +ord(text[s+m-1]) ) % r
            # compute f[s], based on f[s-1]
        f.append(new_fingerprint)# append f[s] to existing f       
    return f

def find_matches_KR(pattern, text, basis=2**16, r=2**32-3):
    """ find all occurances of pattern in text
        using coin flipping Karp-Rabin algorithm """
    if len(pattern) > len(text):
        return []
    p = fingerprint(pattern, basis, r)
    f = text_fingerprint(text, len(pattern), basis, r)
    matches = [s_pos for (s_pos,f_s) in enumerate(f) if f_s == p]
    # list comprehension 
    return matches

In [5]:
text = "abracadabra"
pattern = "abr"

In [6]:
fingerprint("abr")

6422933

In [8]:
base = 2**16
arit = ord("a")*(base**2) + ord("b")*(base**1) + ord("r")*(base**0)
arit
r = 2**32 - 3
fp = arit%r
fp

416618250354

6422933

In [9]:
text_fingerprint(text, 3)

[6422933,
 7471495,
 6357433,
 6488452,
 6357389,
 6553988,
 6357390,
 6422933,
 7471495]

In [10]:
find_matches_KR(pattern, text)

[0, 7]

### Safe version
Makes sure no false positives occur. In the worst case, when all $n-m$ possible substrings are indeed matches, behaves as the naive solution in terms of time complexity.

In [11]:
def find_matches_KR_safe(pattern, text, basis=2**16, r=2**32-3):
    """ a safe version of KR
        checks every suspect for a match """

    if len(pattern) > len(text):
        return []
    p = fingerprint(pattern, basis, r)
    f = text_fingerprint(text, len(pattern), basis, r)
    matches = [s for (s,f_s) in enumerate(f) if f_s == p \
               and text[s:s+len(pattern)]==pattern]
    #note that python performs "short circuit evaluation" of the 'and' statement
    return matches

#### Competition between versions on single char string.
This is the worst-case scenario for the safe version.
Changing $m$ has a greater effect on the safe version than on the standard KR.

In [12]:
import time

text = "a"*1000000
print("text = 'a'*",len(text))
print()
for pattern in ["a"*100, "a"*1000, "a"*10000, "a"*100000]:
    print("pattern = 'a'*",len(pattern))
    for f in [find_matches_KR, find_matches_KR_safe]:
        t0=time.perf_counter()
        res=f(pattern, text)
        t1=time.perf_counter()
        print (f.__name__, t1-t0)
    print("") #newline

text = 'a'* 1000000

pattern = 'a'* 100
find_matches_KR 1.4939075990000106
find_matches_KR_safe 1.6847204460000285

pattern = 'a'* 1000
find_matches_KR 1.3479955149999796
find_matches_KR_safe 1.7813331920000337

pattern = 'a'* 10000
find_matches_KR 1.3464860339999518
find_matches_KR_safe 2.662251118000029

pattern = 'a'* 100000
find_matches_KR 1.2590849160000062
find_matches_KR_safe 12.531647403999955



#### Competition between versions on random strings. 

Note that the standard and safe versions of KR has similar running times. Moreover, as $m$ increases, the running time slightly decreases since there are less candidates to consider.

In [13]:
import random
def gen_str(size):
    ''' Generate a random lowercase English string of length size'''
    s=""
    for i in range(size):
        s+=random.choice("abcdefghijklmnopqrstuvwxyz")
    return s


n=1000000
m=1000
text = gen_str(n)
pattern = gen_str(m)
print("random str of len n=", n, " , random pattern of length m=",m)
for f in [find_matches_KR, find_matches_KR_safe]:
    t0=time.perf_counter()
    f(pattern, text)
    t1=time.perf_counter()
    print (f.__name__, t1-t0)
    

m=10000
pattern = gen_str(m)
print("random str of len n=", n, " , random pattern of length m=",m)
for f in [find_matches_KR, find_matches_KR_safe]:
    t0=time.perf_counter()
    f(pattern, text)
    t1=time.perf_counter()
    print (f.__name__, t1-t0)
    
m=100000
pattern = gen_str(m)
print("random str of len n=", n, " , random pattern of length m=",m)
for f in [find_matches_KR, find_matches_KR_safe]:
    t0=time.perf_counter()
    f(pattern, text)
    t1=time.perf_counter()
    print (f.__name__, t1-t0)

random str of len n= 1000000  , random pattern of length m= 1000


[]

find_matches_KR 1.4612871640000549


[]

find_matches_KR_safe 1.2658764000000247
random str of len n= 1000000  , random pattern of length m= 10000


...

find_matches_KR_safe 1.2447243139999955
random str of len n= 1000000  , random pattern of length m= 100000


[]

find_matches_KR 1.6159478470000295


[]

find_matches_KR_safe 1.2067109560000517


### Choice of $r$

By setting $r$ to be a power of the base we will obtain more false-positives. Specifically, for $r=base$, we will get many more false positives. Why is that?

Note that the computation of the fingerprint is of the form $fp = (x \cdot base + y) \mod r$ for some $x,y$ (where $y$ is the arithmetization of the last character in the current substring). 

In the case where $base = r$ this is equivalent to $(x \cdot r + y) \mod r = y \mod r$, i.e., the fingerprint takes into account only the last character of the substring!

Clearly, choosing $r$ which is a multiple of $base$ is not a good idea. This may also serve as an intuition for why we choose $r$ to be prime - say for example we know that all the fingerprints we compute are even, then if our $r$ is a multiple of $2$, we will always have $fp \mod r$ an even number, and thus we will only utilize half of our range!

In [14]:
# Many false positives
find_matches_KR("da", "abracadabra", basis=2**16, r=2**16)

# Same result here since the "x" will be cancelled out 
find_matches_KR("xa", "abracadabra", basis=2**16, r=2**16)

# The safe check still works, of course
find_matches_KR_safe("da", "abracadabra", basis=2**16, r=2**16)

[2, 4, 6, 9]

[2, 4, 6, 9]

[6]

In [15]:
fingerprint("da", 2**16, r=2**16)
ord("d")*(2**16)**1 + ord("a")
ord("a")

fingerprint("ca", 2**16, r=2**16)
ord("c")*(2**16)**1 + ord("a")
(ord("c")*(2**16)**1 + ord("a") )%2**16

97

6553697

97

97

6488161

97

In [16]:
base = 2**16
r = 2**16
fingerprint("bda", base, r)
ord("b")*(base**2) + ord("d")*(base**1) + ord("a")
(ord("b")*base + ord("d"))*base + ord("a")
((ord("b")*base + ord("d"))*base + ord("a"))%r == ord("a")%r


fingerprint("cda", base, r)
(ord("c")*base + ord("d"))*base + ord("a")
((ord("c")*base + ord("d"))*base + ord("a"))%r == ord("a")%r

97

420913348705

420913348705

True

97

425208316001

True

When $r = base^k$ for some $k$ we have a similar behaviour, though this time it means we only take the last $k$ characters of the rolling hash into account.

In [17]:
find_matches_KR("Humpty", "Humpty Dumpty", r=2**(16*5))

[0, 7]

In [18]:
fingerprint("Humpty", r=2**(16*5))
fingerprint("Dumpty", r=2**(16*5))
fingerprint("Xumpty", r=2**(16*5))

2158299737877522940025

2158299737877522940025

2158299737877522940025

In [19]:
text_fingerprint("Humpty Dumpty", 6, r=2**(16*5))

[2158299737877522940025,
 2010726629729956855840,
 2066067987872461357124,
 2139856371159933386869,
 2232065040410175930477,
 590314951159640293488,
 1254411530052683432052,
 2158299737877522940025]

In [42]:
# Why don't we return the last index?
find_matches_KR("Humpty", "Humpty Dumpty", r=2**(16*6))

[0]

## Question 2  in <a href="http://tau-cs1001-py.wdfiles.com/local--files/home-assignments-2017b/HW6_2017b.pdf"> this exercise</a>

In [20]:
def fingerprint(string, basis=2**16, r=2**32-3):
    """ used to computes karp-rabin fingerprint of the pattern
    employs Horner method (modulo r) """
    partial_sum = 0
    for x in string:
        partial_sum = (partial_sum*basis + ord(x)) % r
    return partial_sum

def slide(prev_fp, prev_char, next_char, b_power, basis=2**16, r=2**32-3):
    new_fp=((prev_fp - ord(prev_char)*b_power)*basis + ord(next_char)) % r
    return new_fp

### Section (a)

Build a generator which, given a string text and a length parameter generates all KR fingerprints of desired length in text.
Instructions:
* Make use of both $fingerprint$ and $slide$
* The generator can call $fingerprint$ only once


In [21]:
def kr_gen(text, length, basis=2**16, r=2**32-3):
    fp = fingerprint(text[:length])
    yield fp
    b_power = pow(basis, length - 1, r)
    for s in range(1, len(text) - length + 1):
        fp = slide(fp, text[s - 1], text[s - 1 + length], b_power)
        yield fp

In [22]:
gen = kr_gen("abracadabra", 3)
next(gen)
next(gen)


6422933

7471495

In [23]:
next(gen)
next(gen)
next(gen)
next(gen)
next(gen)
next(gen)
next(gen)
next(gen)
next(gen)
next(gen)

6357433

6488452

6357389

6553988

6357390

6422933

7471495

StopIteration: 

### Section (b)

Build a generator which, given two strings $text1, text2$ and a length parameter $\ell$ generates all pairs of indices $(i,j)$ such that $text1[i:i+\ell] == text2[j:j+\ell]$.

Instructions:
* Use $kr\_gen$ from the previous section
* The generator must work in space smaller than the size of the strings. In particular, at no point can you save the array of fingerprints of any entire string

In [24]:
def generate_shared_substrings(text1, text2, length):
    g1 = kr_gen(text1, length)
    i1 = 0
    for fp1 in g1:
        g2 = kr_gen(text2, length)
        i2 = 0
        for fp2 in g2:
            if fp1 == fp2:
                yield(i1, i2)
            i2 += 1
        i1 += 1

In [25]:
g = generate_shared_substrings("abcdef","xcdefx",3)
next(g)
next(g)

(2, 1)

(3, 2)

In [51]:
next(g)

StopIteration: 